# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. It consists of the 500 biggest companies in the US and is market capitalization-weighted. This means the companies are weighted by size in the index (larger companies have higher weight). The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management. 

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
import yfinance as yf


## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('../data/S_P_500.csv')
stocks

,Index,Company,Symbol,Weight,Price,Change,Percent Change
0,1,Microsoft Corp,MSFT,6.78%,438.73,0.56,(0.13%)
1,2,Apple Inc.,AAPL,6.17%,198.53,1.04,(0.53%)
2,3,Nvidia Corp,NVDA,5.98%,116.65,-0.72,(-0.61%)
3,4,Amazon.com Inc,AMZN,3.77%,193.06,0.98,(0.51%)
4,5,"Meta Platforms, Inc. Class A",META,2.73%,592.49,-5.52,(-0.92%)
...,...,...,...,...,...,...,...
498,499,Fox Corporation Class B,FOX,0.01%,46.72,-0.24,(-0.51%)
499,500,Apa Corporation,APA,0.01%,16.72,0.54,(3.34%)
500,501,"Caesars Entertainment, Inc.",CZR,0.01%,28.03,0.20,(0.72%)
501,502,"Mohawk Industries, Inc.",MHK,0.01%,106.31,1.20,(1.14%)


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

Note: if the next kernel does not work, ensure that secrets.py is not part of an existing library yet (if so, rename the file) and restart the kernels.

### IEX out of business
The token used in this course is IEX, however in 2024 this has been discontinued. For this course mock data will be used. Once through the course, and based on other tutorials we can choose alternative API broker systems (paid):
    - Freetrade.io
    -IBKR
    -polygon.io

Or free (yahoo finance api - try here)

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [ ]:
# Market Capitalization for AAPL
# Get the ticker object for Apple Inc.
# You can replace "AAPL" with any other stock ticker symbol
ticker = yf.Ticker("AAPL")

# Access the info dictionary
info = ticker.info

# Get the market capitalization
market_cap = info.get("marketCap")

# Print the market cap
print(f"Market Capitalization of AAPL: {market_cap}")

Market Capitalization of AAPL: 3118296203264


In [7]:
# Get the current price
current_price = ticker.history(period="1d")["Close"].iloc[-1]
print(f"Current Price of AAPL: {current_price}")

Current Price of AAPL: 208.77999877929688


## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: